In [ ]:
%pip install ccrvam --upgrade

> Make sure to have ccrvam's latest version installed using `pip`. More information about the latest version can be found at https://pypi.org/project/ccrvam/

In [ ]:
import numpy as np
from ccrvam import (
    best_subset_ccram,
    all_subsets_ccram
)
from ccrvam import DataProcessor

# 2-Dimensional Case 

### Create Sample Contingency Table

For a 2D contingency table:

- `axis=0`: First variable ($X_1$) with 5 categories
- `axis=1`: Second variable ($X_2$) with 3 categories

The axis indexing follows NumPy's convention, starting from the outermost dimension. The variables are ordered such that:

- $X_1$ corresponds to (rows)
- $X_2$ corresponds to (columns)

This ordering is important for calculating measures of regression association between two variables.

In [ ]:
contingency_table = np.array([
    [0, 0, 20],
    [0, 10, 0],
    [20, 0, 0],
    [0, 10, 0],
    [0, 0, 20]
])

### Calculate All Subsets CCRAM and Find Best Subset

For a 2D contingency table, we can calculate CCRAM for predicting $X_2$ (response) from $X_1$ (predictor). Since there's only one possible predictor, this demonstrates the basic functionality.

In [ ]:
# Calculate all subsets CCRAM for predicting X2 (axis 2) from X1 (axis 1)
result_2d = all_subsets_ccram(
    contingency_table=contingency_table,
    response=2,  # Predict X2 (columns)
    scaled=False  # Use CCRAM (not scaled)
)

print("=== All Subsets CCRAM Results (2D) ===")
print(result_2d.results_df)
print("\n=== Summary Statistics ===")
print(result_2d.summary())


### Find the Best Subset

The `best_subset_ccram` function returns the optimal predictor subset with the highest (S)CCRAM value.


In [ ]:
# Find the best subset for predicting X2
best_2d = best_subset_ccram(
    contingency_table=contingency_table,
    response=2,
    scaled=False
)

print("=== Best Subset Result (2D) ===")
print(best_2d)
print("\n=== Best Subset Summary DataFrame ===")
print(best_2d.summary_df())


### Compare CCRAM vs Scaled CCRAM (SCCRAM)

The scaled version (SCCRAM) normalizes the CCRAM value to account for the maximum possible association.

In [ ]:
# Compare with Scaled CCRAM (SCCRAM)
result_2d_scaled = all_subsets_ccram(
    contingency_table=contingency_table,
    response=2,
    scaled=True  # Use SCCRAM
)

print("=== All Subsets SCCRAM Results (2D) ===")
print(result_2d_scaled.results_df)
print(f"\nCCRAM value: {result_2d.results_df['ccram'].iloc[0]:.6f}")
print(f"SCCRAM value: {result_2d_scaled.results_df['sccram'].iloc[0]:.6f}")


# 4-Dimensional Case (Real Data Analysis)

### Load Sample Data in Cases / Frequency Form

This example demonstrates how one can analyze relationships between multiple categorical variables in a clinical dataset of back pain treatments using the `DataProcessor`.

The dataset contains 4 categorical variables from a medical study:

| Variable | Description | Categories |
|----------|-------------|------------|
| X₁ | Length of Previous Attack | 1=Short, 2=Long |
| X₂ | Pain Change | 1=Better, 2=Same, 3=Worse |
| X₃ | Lordosis | 1=Absent/Decreasing, 2=Present/Increasing |
| Pain | Back Pain Outcome | worse (W)=1, same (S)=2, slight.improvement (SI)=3, moderate.improvement (MODI)=4, marked.improvement (MARI)=5, complete.relief (CR)=6 |

1. Define variable names, and dimension tuple (the number of categores of each variable according to the order of the variables defined) 
2. (Optional) If your data has any variables with category names that are not integers, then create a category mapping for non-integer categories (for instance, required for 'pain' variable in the above dataset)
3. Load case-form/freq-form data from file (from path provided as an argument) or type table-form with proper mappings into a contingency table

Citation for the above dataset:
- J. A. Anderson, Regression and ordered categorical variables, Journal of the Royal Statistical Society: Series B (Methodological) 46 (1984)
1–22.

In [ ]:
var_list_4d = ["x1", "x2", "x3", "pain"]
category_map_4d = {
    "pain": {
        "worse": 1,
        "same": 2,
        "slight.improvement": 3,
        "moderate.improvement": 4,
        "marked.improvement": 5,
        "complete.relief": 6
    },
}
data_dimension = (2, 3, 2, 6)

rda_contingency_table = DataProcessor.load_data(
                        "./data/caseform.pain.txt",
                        data_form="case_form",
                        dimension=data_dimension,
                        var_list=var_list_4d,
                        category_map=category_map_4d,
                        named=True,
                        delimiter="\t"
                    )
print("Read contingency table from case form data!")
print(rda_contingency_table)

rda_contingency_table_from_freq = DataProcessor.load_data(
                        "./data/freqform.pain.txt",
                        data_form="frequency_form",
                        dimension=data_dimension,
                        var_list=var_list_4d,
                        category_map=category_map_4d,
                        named=True,
                        delimiter="\t"
                    )
print("Read contingency table from frequency form data!")
print(rda_contingency_table_from_freq)

### Calculate All Subsets CCRAM for Predicting Pain Outcome

With 4 dimensions, we have 3 potential predictors ($X_1$, $X_2$, $X_3$) for predicting the Pain outcome ($X_4$). This gives us $\binom{3}{1} + \binom{3}{2} + \binom{3}{3} = 7$ possible predictor subsets.

In [ ]:
# Define variable names for better readability in output
variable_names_4d = {
    1: "PrevAttack",
    2: "PainChange", 
    3: "Lordosis",
    4: "Pain"
}

# Calculate all subsets CCRAM for predicting Pain (variable 4)
result_4d = all_subsets_ccram(
    contingency_table=rda_contingency_table,
    response=4,  # Predict Pain outcome
    scaled=False,
    variable_names=variable_names_4d
)

print("=== All Subsets CCRAM Results (4D) ===")
print(result_4d.results_df)
print("\n=== Summary Statistics by Number of Predictors ===")
print(result_4d.summary())


### Find the Best Predictor Subset for Pain Outcome

The `best_subset_ccram` function identifies which combination of predictors yields the highest CCRAM value.


In [ ]:
# Find the best subset for predicting Pain
best_4d = best_subset_ccram(
    contingency_table=rda_contingency_table,
    response=4,
    scaled=False,
    variable_names=variable_names_4d
)

print("=== Best Subset Result (4D) ===")
print(best_4d)
print("\n=== Best Subset Summary DataFrame ===")
print(best_4d.summary_df())


### Get Top Subsets and Filter by k

The `SubsetCCRAMResult` object provides helper methods to explore the results.


In [ ]:
# Get top 3 subsets across all k values
print("=== Top 3 Subsets Overall ===")
print(result_4d.get_top_subsets(n=3))

# Get all subsets with exactly k=2 predictors
print("\n=== All Subsets with k=2 Predictors ===")
print(result_4d.get_subsets_by_k(k=2))


### Find Best Subset with Fixed k

You can also find the best subset constrained to a specific number of predictors using the `k` parameter.


In [ ]:
# Find best subset with exactly k=1 predictor
best_k1 = best_subset_ccram(
    contingency_table=rda_contingency_table,
    response=4,
    scaled=False,
    k=1,  # Constrain to single predictor
    variable_names=variable_names_4d
)

print("=== Best Single Predictor (k=1) ===")
print(best_k1)

# Find best subset with exactly k=2 predictors
best_k2 = best_subset_ccram(
    contingency_table=rda_contingency_table,
    response=4,
    scaled=False,
    k=2,  # Constrain to two predictors
    variable_names=variable_names_4d
)

print("\n=== Best Two-Predictor Subset (k=2) ===")
print(best_k2)


### Compare CCRAM vs SCCRAM for 4D Case

Finally, we compare the unscaled and scaled versions of CCRAM to see how normalization affects the results.


In [ ]:
# Calculate SCCRAM for all subsets
result_4d_scaled = all_subsets_ccram(
    contingency_table=rda_contingency_table,
    response=4,
    scaled=True,  # Use SCCRAM
    variable_names=variable_names_4d
)

print("=== All Subsets SCCRAM Results (4D) ===")
print(result_4d_scaled.results_df)

# Find best subset using SCCRAM
best_4d_scaled = best_subset_ccram(
    contingency_table=rda_contingency_table,
    response=4,
    scaled=True,
    variable_names=variable_names_4d
)

print("\n=== Best Subset using SCCRAM ===")
print(best_4d_scaled)
